In [1]:
%load_ext beam_setup

✨ | Setting up the Beam environment for interactive use
🚀 | Standard modules will be automatically imported so you can use them without explicit import
🛸 | Beam library is loaded from path: /home/dayosupp/elad/beamds/beam
🔥 | 10:07:51 (0:00:03.632311) | INFO     🗎 Beam logger (2.7.6): logs are saved to /root/beam_data/logs/ipykernel_launcher-20250206-100751.log (∫__init__.py:__getattr__-#143)
⏲ | Done importing packages. It took:  4.0 seconds


# Connect to the Elasticsearch endpoint

In Beam, elasticsearch endpoint is considered as a resource that can be connected via URI string, just like storages, llms, http servers etc.
We use resource to initiate the client

In [4]:
es = resource('elastic://10.0.7.228:31063')

Lets ping to see its alive

In [12]:
es.ping()

True

We use Pathlib API to navigate between the different indices/alias

In [13]:
list(es)

/home/dayosupp/elad/beamds/notebooks/../beam/docs/elastic.py:499: ElasticsearchWarning: this request accesses system indices: [.kibana_8.16.1_001, .kibana_ingest_8.16.1_001, .kibana_task_manager_8.16.1_001, .apm-custom-link, .kibana_security_solution_8.16.1_001, .apm-agent-configuration, .kibana_analytics_8.16.1_001, .async-search, .kibana_alerting_cases_8.16.1_001, .kibana_usage_counters_8.16.1_001], but in a future major version, direct access to system indices will be prevented by default
  for ind, av in self.client.indices.get_alias(index=wildcard).items():


[elastic://10.0.7.228:31063/historical_prices]

In [85]:
ind = es.joinpath('historical_prices')

Lets explore the schema

In [18]:
ind.schema

{'capital_gains': {'type': 'float'},
 'close': {'type': 'float'},
 'date': {'type': 'date'},
 'dividends': {'type': 'float'},
 'high': {'type': 'float'},
 'interval': {'type': 'text',
  'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
 'low': {'type': 'float'},
 'open': {'type': 'float'},
 'period': {'type': 'text',
  'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
 'processed_at': {'type': 'date'},
 'stock_splits': {'type': 'float'},
 'ticker': {'type': 'text',
  'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
 'version': {'type': 'keyword'},
 'volume': {'type': 'long'}}

Now we can start using the pandas like API, with distinct difference, ind may contain also query information which is applied before any operation

In [15]:
ind.count()

39602621

adding query changes the result

In [17]:
from beam.docs.queries import TimeFilter

In [21]:
q = ind & TimeFilter(field='date', start=datetime(2023,1,1), end=datetime(2024,1,1))

In [22]:
q

elastic://10.0.7.228:31063/historical_prices | query: date between 2023-01-01 00:00:00 and 2024-01-01 00...

In [24]:
q.count()

2041601

We can use KQL queries as well

In [25]:
(ind & "ticker: msft").count()

9784

and we can retrieve samples

In [186]:
(ind & "ticker: msft").head()

,date,open,high,low,close,volume,dividends,stock_splits,interval,period,start,end,ticker,processed_at,version
Ik_zR5QBsuJ1Mck3OIZV,2007-07-11T00:00:00-04:00,21.600716,21.903598,21.578553,21.785400,48017000,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
LE_zR5QBsuJ1Mck3OIZV,2007-07-25T00:00:00-04:00,22.893506,23.122515,22.435489,22.686659,54950100,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
O0_zR5QBsuJ1Mck3OIZV,2007-08-15T00:00:00-04:00,20.935100,21.491095,20.794247,20.831314,48117700,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
UE_zR5QBsuJ1Mck3OIZV,2007-09-14T00:00:00-04:00,21.483679,21.580053,21.409546,21.528160,33496600,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
eU_zR5QBsuJ1Mck3OIZV,2007-11-12T00:00:00-05:00,24.701033,24.990151,24.478636,24.745514,84865200,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2


and even randomly sample data

In [197]:
(ind & "ticker: msft").sample(5)

,date,open,high,low,close,volume,dividends,stock_splits,interval,period,start,end,ticker,processed_at,version
Qk_zR5QBsuJ1Mck3OZ8L,2021-07-15T00:00:00-04:00,282.000000,282.510010,279.829987,281.029999,22604200,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
CU_zR5QBsuJ1Mck3OJCk,2013-09-30T00:00:00-04:00,28.189195,28.454004,27.932930,28.428375,39839500,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
Nk_zR5QBsuJ1Mck3OIlw,2010-08-25T00:00:00-04:00,18.877972,19.051020,18.775717,18.956631,47404800,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
_0_zR5QBsuJ1Mck3OZ8L,2022-04-13T00:00:00-04:00,282.730011,288.579987,281.299988,287.619995,21907200,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
q0_zR5QBsuJ1Mck3OH8G,2002-11-12T00:00:00-05:00,17.141247,17.562649,17.077879,17.271152,81398600,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2


We can use standard pandas operations

In [192]:
ind.loc[['YU_zR5QBsuJ1Mck3OJzx', '7U_zR5QBsuJ1Mck3N3vN']].as_df()

,date,open,high,low,close,volume,dividends,stock_splits,interval,period,start,end,ticker,processed_at,version
7U_zR5QBsuJ1Mck3N3vN,1999-01-21T00:00:00-05:00,25.624737,25.872271,24.991050,25.080162,80077200,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2
YU_zR5QBsuJ1Mck3OJzx,2020-07-14T00:00:00-04:00,204.153263,206.847180,200.092575,206.351974,37591800,0.0,0.0,1d,max,None,None,msft,2025-01-08T21:26:55.985461,0.0.2


In [193]:
ind['ticker'].nunique()

6234

In [194]:
ind['ticker'].value_counts()

ip       31726
cnp      31726
ge       31726
dis      31726
pg       31726
         ...  
aspc         4
mask         3
cepo         2
zybt         2
mficl        1
Length: 6227, dtype: int64

In [195]:
(ind & 'ticker: tsla')[['open', 'high', 'low', 'close']].mean()

low      80.050248
high     83.779151
open     81.974180
close    81.957195
Name: avg, dtype: float64

## Groupby

we can even use the pandas groupby syntax, lets see what was the average price for all the stocks that starts with "ms" in 2023

In [198]:
q = ind & 'ticker: ms*' & TimeFilter(field='date', start=datetime(2023,1,1), end=datetime(2024,1,1))

In [205]:
q.groupby('ticker').agg({'close': 'mean', 'open': 'mean', 'high': 'mean', 'low': 'mean'}).as_df().head()

False


,count,close_avg,open_avg,high_avg,low_avg
ticker,,,,,
ms,500,86.393280,86.33932,87.232400,85.515480
msa,500,153.166440,153.12868,154.774920,151.593840
msb,500,20.904000,20.84936,21.377480,20.477760
msc,500,6.448520,6.42772,6.669480,6.200880
msci,500,514.673161,514.19232,520.167159,508.615319


we can aggregate over more than one field

In [224]:
q.groupby(['ticker', 'interval']).agg({'close': 'mean', 'open': 'mean', 'high': 'mean', 'low': 'mean'}).as_df().head()

,,count,close_avg,open_avg,high_avg,low_avg
ticker,interval,,,,,
ms,1d,500,86.393280,86.339320,87.232400,85.515480
msa,1d,500,153.166440,153.128680,154.774920,151.593840
msai,1d,250,10.375096,10.390216,10.485096,10.310928
msb,1d,500,20.904000,20.849360,21.377480,20.477760
msbi,1d,250,22.633160,22.631920,22.938600,22.329556


## Bonus: use LLM to explore the DB and answer questions

In [238]:
llm = resource("openai:///gpt-4o")

In [248]:
r = ind.ask("did msft trade over 100$ in 2023 and when?", llm=llm, execute=True, answer=True)

The LLM will produce a DSL query, execute it and iterprate the results to provide the final answer

In [251]:
r.query

Bool(must=[Term(ticker__keyword='msft'), Range(date={'gte': '2023-01-01', 'lte': '2023-12-31'}), Range(high={'gt': 100})])

In [252]:
r.df.head()

,date,high
w0_zR5QBsuJ1Mck3OaAq,2023-01-25T00:00:00-05:00,243.300003
xE_zR5QBsuJ1Mck3OaAq,2023-01-26T00:00:00-05:00,248.309998
x0_zR5QBsuJ1Mck3OaAq,2023-01-31T00:00:00-05:00,247.949997
yE_zR5QBsuJ1Mck3OaAq,2023-02-01T00:00:00-05:00,255.179993
yU_zR5QBsuJ1Mck3OaAq,2023-02-02T00:00:00-05:00,264.690002


In [249]:
print(r.text_answer)

Yes, MSFT traded over $100 in 2023. The data shows that the highest trading price for MSFT in 2023 was $384.30, and it consistently traded above $100 throughout the year. Here are some sample dates when MSFT traded over $100:

- May 8, 2023, with a high of $310.20
- February 16, 2023, with a high of $266.74
- July 3, 2023, with a high of $340.90
- July 19, 2023, with a high of $362.46
- December 14, 2023, with a high of $373.76

These are just a few examples, and the data indicates that MSFT traded over $100 on all recorded dates in 2023.


# Conclusion

Elasticsearch API can be complicated and cumbersome, 

beam 🔥 gets you covered and lets you enjoy all the elasticsearch advantages 🚀 with your preferred pandas API 🐼